In [ ]:
import openai
from os import environ

# -*- Set OPENAI_API_KEY in one of 3 ways:
# 1. In `workspace/secrets/openai-api-key.txt`
# 2. Set OPENAI_API_KEY Env variable in workspace/secrets/dev_jupyter_secrets.yml
#    NOTE: needs a container restart using `phi ws restart dev:docker:lab -y`
# 3. Directly in this notebook (NOT RECOMMENDED)
if environ.get("OPENAI_API_KEY", None) is None:
    from workspace.settings import ws_settings

    OPENAI_API_KEY_FILE = ws_settings.ws_root.joinpath(
        "workspace/secrets/openai-api-key.txt"
    )
    openai.api_key = OPENAI_API_KEY_FILE.read_text().strip()

In [ ]:
from textwrap import dedent

from phidata.utils.log import logger
from phidata.llm.duckdb.agent import create_duckdb_agent
from phidata.llm.duckdb.connection import create_duckdb_connection
from phidata.llm.duckdb.loader import load_s3_path_to_table
from phidata.llm.duckdb.query import run_duckdb_query

In [ ]:
# -*- Create a DuckDB connection
duckdb_connection = create_duckdb_connection()

# -*- Create a DuckDB agent
duckdb_agent = create_duckdb_agent(duckdb_connection=duckdb_connection)

# -*- List of test datasets
Tables = {
    "titanic": "s3://phidata-public/demo_data/titanic.csv",
    "census": "s3://phidata-public/demo_data/census_2017.csv",
    "covid": "s3://phidata-public/demo_data/covid_19_data.csv",
    "air_quality": "s3://phidata-public/demo_data/air_quality.csv",
}

In [ ]:
# -*- Set table and query
table: str = "titanic"
s3_data_path = Tables[table]
query = "How many passengers survived by gender? show me the SQL"

# -*- Add an initial system message
duckgpt_messages = [
    {
        "role": "system",
        "content": """You are a helpful assistant that answers natural language questions by querying data using duckdb""",  # noqa: E501
    },
]

# -*- Load data to duckdb
_loaded_table_name, executed_query = load_s3_path_to_table(
    duckdb_connection, s3_data_path
)
logger.info(f"Table: {_loaded_table_name} loaded")

# -*- Add executed_query to messages
duckgpt_messages.append(
    {
        "role": "system",
        "content": dedent(
            """\
        Startup SQL Queries:
        ```
        {}
        ```
    """.format(
                executed_query
            )
        ),
    },
)

# -*- Add user query
duckgpt_messages.append({"role": "user", "content": query})

# -*- Create input for agent
inputs = {
    "input": duckgpt_messages,
    "table_names": run_duckdb_query(duckdb_connection, "show tables"),
}

# -*- Generate response
result = duckdb_agent(inputs)

# -*- Get the output
if "output" in result:
    logger.info(result["output"])
else:
    logger.error("Could not understand")